**Histórico dos dados dos fundos de investimentos**
* https://dados.cvm.gov.br/dataset/fi-doc-cda

---
O arquivo zipado da CVM contém vários arquivos excel. Os arquivos que eu tenho interesse são:

* cda_fi_BLC_1: arquivo que contém os 'Títulos Públicos';
* cda_fi_BLC_2: arquivo que contém as 'Cotas de Fundos';
* cda_fi_BLC_4: arquivo que contém as 'Ações', 'BDRs', 'Opções', 'Mercado futuro', 'Debêntures', 'Obrigações por ações' (posição vendida);
* cda_fi_BLC_7: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_BLC_8: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_PL: arquivo que contém o PL dos fundos de investimentos.

**Qual é o objetivo desse notebook?**
* Criar um arquivo excel para cada fundo de investimento, onde cada classe de ativo estará em uma aba diferente.



In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import zipfile

## Extraindo os arquivos zipados

In [2]:
# Mês e ano escolhido
mes = '04'
ano = '2023'

# Nome do arquivo zipad
zip_filename = f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm_zip//cda_fi_{ano}{mes}.zip'

# # Visualizando a lista dos nomes dos arquivos dentro do arquivo zipado
# with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
#     print("Arquivos dentro do arquivo zipado:")
#     print(zip_ref.namelist())

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    f'cda_fi_BLC_1_{ano}{mes}.csv', 
    f'cda_fi_BLC_2_{ano}{mes}.csv', 
    f'cda_fi_BLC_4_{ano}{mes}.csv', 
    f'cda_fi_BLC_7_{ano}{mes}.csv', 
    f'cda_fi_BLC_8_{ano}{mes}.csv',
    f'cda_fi_PL_{ano}{mes}.csv'
]

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extrai os arquivos selecionados para um diretório específico
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}')


## Funções

In [4]:
def open_cda_1(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_1'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_1'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'TP_TITPUB', 'DT_VENC']]

    # Mesclando as colunas 'TP_TITPUB' e 'DT_VENC' em apenas em uma coluna
    df['TP_TITPUB'] = df['TP_TITPUB'] + ' ' + df['DT_VENC']

    # Removendo a coluna 'DT_VENC'
    df = df.drop('DT_VENC', axis=1)

    # Renomeando a coluna 'TP_TITPUB' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_2/4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'TP_TITPUB':'CD_ATIVO'})

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_2(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_2'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_2'

    """
    # Lendo o arquivo. Adicionei o 'low_memory=False' para não dar o aviso -> DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1', low_memory=False)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'NM_FUNDO_COTA']]

    # Renomeando a coluna 'NM_FUNDO_COTA' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'NM_FUNDO_COTA':'CD_ATIVO'})

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_4(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_4'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_4'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'CD_ATIVO']]

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_7(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_7'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_7'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas.
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'EMISSOR']]

    # Renomeando a coluna 'EMISSOR' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs.
    df.rename(columns={"EMISSOR": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_8(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_8'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_8'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DS_ATIVO']]

    # Renomeando a coluna 'DS_ATIVO' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs
    df.rename(columns={"DS_ATIVO": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    # Selecionando apenas o ativo 'BDR', porque neste arquivo também possui um ativo chamado 'Títulos Públicos', mas não é o principal 'Títulos Públicos', que está no 'cda_fi_BLC_1'
    filt_bdr = (df['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    df = df.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    
    return df


def pl_fundo(path: str, cnpj: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_PL'.
    
    Paramenters:
    path (str): caminho do arquivo.
    cnpj (str): cnpj do fundo de investimento que você está procurando.

    Returns:
    Dataframe com o valor do patrimônio líquido do fundo de investimentos específico.

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando o fundo de investimentos específicos
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # # Transformando os dtypes da coluna
    # fundo_espec['VL_PATRIM_LIQ'] = fundo_espec.loc[:, 'VL_PATRIM_LIQ'].astype(float)

    return fundo_espec['VL_PATRIM_LIQ']


def fundo_cnpj(cnpj: str) -> DataFrame:
    """
    Separando o df do fundo de investimentos em várias categorias.

    Parameters:
    cnpj (str): cnpj do fundo de investimento que você está procurando.

    Returns:
    Vários dataframes de categorias diferentes: ações, BDRs, investimentos no exterior, cotas de fundos e títulos públicos.
    
    """
    # Lendo o df concatenado
    filt_cnpj = df_ativos['CNPJ_FUNDO'] == cnpj
    fundo_espec = df_ativos.loc[filt_cnpj]

    # Ações
    filt_acoes = (fundo_espec['TP_APLIC'] == 'Ações')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_acoes = fundo_espec.loc[filt_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_acao = lambda x: (x / df_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_acoes['PORCENTAGEM'] = list(map(porcentagem_acao, df_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_acoes = df_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # BDRs
    filt_bdr = (fundo_espec['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_bdr = fundo_espec.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_bdr = lambda x: (x / df_bdr['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_bdr['PORCENTAGEM'] = list(map(porcentagem_bdr, df_bdr['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_bdr =  df_bdr.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Investimentos Exterior
    filt_exterior = (fundo_espec['TP_APLIC'] == 'Investimento no Exterior')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_exterior = fundo_espec.loc[filt_exterior].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_exterior = lambda x: (x / df_exterior['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_exterior['PORCENTAGEM'] = list(map(porcentagem_exterior, df_exterior['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_exterior = df_exterior.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Cotas de Fundos
    filt_cotas_fundos = (fundo_espec['TP_APLIC'] == 'Cotas de Fundos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_cotas_fundos = fundo_espec.loc[filt_cotas_fundos].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada cota de fundo
    porcentagem_cotas = lambda x: (x / df_cotas_fundos['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_cotas_fundos['PORCENTAGEM'] = list(map(porcentagem_cotas, df_cotas_fundos['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_cotas_fundos = df_cotas_fundos.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Títulos públicos
    filt_titulos_pub = (fundo_espec['TP_APLIC'] == 'Títulos Públicos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_titulos_pub = fundo_espec.loc[filt_titulos_pub].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_titulos = lambda x: (x / df_titulos_pub['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_titulos_pub['PORCENTAGEM'] = list(map(porcentagem_titulos, df_titulos_pub['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_titulos_pub = df_titulos_pub.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Obrigações por ações e outros TVM recebidos em empréstimo
    filt_vendido_acoes = (fundo_espec['TP_APLIC'] == 'Obrigações por ações e outros TVM recebidos em empréstimo')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_vendido_acoes = fundo_espec.loc[filt_vendido_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_vendido = lambda x: (x / df_vendido_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_vendido_acoes['PORCENTAGEM'] = list(map(porcentagem_vendido, df_vendido_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_vendido_acoes = df_vendido_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    return (
        df_acoes, 
        df_bdr, 
        df_exterior, 
        df_cotas_fundos, 
        df_titulos_pub, 
        df_vendido_acoes
    )

In [5]:
# Aplicando as funções para formatar os arquivos excel
df_cda_1 = open_cda_1(path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_1_202304.csv')
df_cda_2 = open_cda_2(path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_2_202304.csv')
df_cda_4 = open_cda_4(path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_4_202304.csv')
df_cda_7 = open_cda_7(path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_7_202304.csv')
df_cda_8 = open_cda_8(path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_8_202304.csv')

# Concatenando os dfs
df_ativos = pd.concat([df_cda_1, df_cda_2, df_cda_4, df_cda_7, df_cda_8])
df_ativos.tail()

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,VL_MERC_POS_FINAL,CD_ATIVO
120412,FI,34.658.753/0001-00,DAYCOVAL FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍ...,2023-04-30,Brazilian Depository Receipt - BDR,BDR nível I,82.35,MACY DRN
62598,FI,13.089.341/0001-27,FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRI...,2023-04-30,Brazilian Depository Receipt - BDR,BDR nível III,48.60,PPLA11
132178,FI,37.553.253/0001-00,NIMROD FUNDO DE INVESTIMENTO MULTIMERCADO CRÉD...,2023-04-30,Brazilian Depository Receipt - BDR,BDR nível I,17.58,INBR/INBR32/BRINBRBDR007
83310,FI,20.895.433/0001-60,TERA FUNDO DE INVESTIMENTO MULTIMERCADO INVEST...,2023-04-30,Brazilian Depository Receipt - BDR,BDR nível I,8.58,MILK33 - LAEP - MILK33
167256,FI,46.997.509/0001-51,ESH THETA MASTER FUNDO DE INVESTIMENTO MULTIME...,2023-04-30,Brazilian Depository Receipt - BDR,BDR nível I,0.04,MILK33 - LAEP - MILK33


In [6]:
# Quais são os tipos de ativos o fundo de investimento possui
lista_ativos = df_ativos['TP_APLIC'].drop_duplicates().to_list()
lista_ativos

['Operações Compromissadas',
 'Títulos Públicos',
 'Cotas de Fundos',
 'Ações',
 'Debêntures',
 'Certificado ou recibo de depósito de valores mobiliários',
 'Outros valores mobiliários registrados na CVM objeto de oferta pública',
 'Ações e outros TVM cedidos em empréstimo',
 'Brazilian Depository Receipt - BDR',
 'Mercado Futuro - Posições compradas',
 'Mercado Futuro - Posições vendidas',
 'Opções - Posições lançadas',
 'Opções - Posições titulares',
 'Obrigações por ações e outros TVM recebidos em empréstimo',
 'Compras a termo a receber',
 'Vendas a termo a receber',
 'Investimento no Exterior']

## Fundos de Investimentos

### Fundo Verde

* https://maisretorno.com/fundo/verde-am-long-bias-master-fia/carteira

In [7]:
verde_acoes, verde_bdr, verde_exterior, verde_cotas_fundos, verde_titulos_pub, verde_vendido_acoes = fundo_cnpj(cnpj='16.929.553/0001-63')
verde_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='16.929.553/0001-63'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(verde_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_bdr = round(pd.Series(verde_bdr['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_exterior = round(pd.Series(verde_exterior['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(verde_cotas_fundos['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_titulos_pub = round(pd.Series(verde_titulos_pub['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(verde_vendido_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
verde_acoes = pd.concat([verde_acoes, df_linha_pct_acoes])

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
verde_bdr = pd.concat([verde_bdr, df_linha_pct_bdr])

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
verde_exterior = pd.concat([verde_exterior, df_linha_pct_exterior])

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
verde_cotas_fundos = pd.concat([verde_cotas_fundos, df_linha_pct_cotas_fundos])

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
verde_titulos_pub = pd.concat([verde_titulos_pub, df_linha_pct_titulos_pub])

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
verde_vendido_acoes = pd.concat([verde_vendido_acoes, df_linha_pct_vendido_acoes ])

In [8]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//verde//fundo_verde_{ano}{mes}.xlsx') as writer:
    verde_acoes.to_excel(writer, sheet_name='acoes')
    verde_bdr.to_excel(writer, sheet_name='bdr')
    verde_exterior.to_excel(writer, sheet_name='exterior')
    verde_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    verde_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    verde_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

### Fundo Dynamo

* https://maisretorno.com/fundo/dynamo-cougar-master-fia/carteira

In [9]:
dynamo_acoes, dynamo_bdr, dynamo_exterior, dynamo_cotas_fundos, dynamo_titulos_pub, dynamo_vendido_acoes = fundo_cnpj(cnpj='37.916.879/0001-26')
dynamo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='37.916.879/0001-26'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(dynamo_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_bdr = round(pd.Series(dynamo_bdr['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_exterior = round(pd.Series(dynamo_exterior['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(dynamo_cotas_fundos['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_titulos_pub = round(pd.Series(dynamo_titulos_pub['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(dynamo_vendido_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
dynamo_acoes = pd.concat([dynamo_acoes, df_linha_pct_acoes])

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
dynamo_bdr = pd.concat([dynamo_bdr, df_linha_pct_bdr])

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
dynamo_exterior = pd.concat([dynamo_exterior, df_linha_pct_exterior])

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
dynamo_cotas_fundos = pd.concat([dynamo_cotas_fundos, df_linha_pct_cotas_fundos])

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
dynamo_titulos_pub = pd.concat([dynamo_titulos_pub, df_linha_pct_titulos_pub])

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
dynamo_vendido_acoes = pd.concat([dynamo_vendido_acoes, df_linha_pct_vendido_acoes])

In [10]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//dynamo//fundo_dynamo_{ano}{mes}.xlsx') as writer:
    dynamo_acoes.to_excel(writer, sheet_name='acoes')
    dynamo_bdr.to_excel(writer, sheet_name='bdr')
    dynamo_exterior.to_excel(writer, sheet_name='exterior')
    dynamo_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    dynamo_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    dynamo_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

### Fundo IP

* https://maisretorno.com/fundo/ip-participacoes-master-fia-bdr-nivel-i/carteira

In [11]:
ip_acoes, ip_bdr, ip_exterior, ip_cotas_fundos, ip_titulos_pub, ip_vendido_acoes = fundo_cnpj(cnpj='11.435.298/0001-89')
ip_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='11.435.298/0001-89'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(ip_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_bdr = round(pd.Series(ip_bdr['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_exterior = round(pd.Series(ip_exterior['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(ip_cotas_fundos['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_titulos_pub = round(pd.Series(ip_titulos_pub['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(ip_vendido_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
ip_acoes = pd.concat([ip_acoes, df_linha_pct_acoes])

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
ip_bdr = pd.concat([ip_bdr, df_linha_pct_bdr])

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
ip_exterior = pd.concat([ip_exterior, df_linha_pct_exterior])

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
ip_cotas_fundos = pd.concat([ip_cotas_fundos, df_linha_pct_cotas_fundos])

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
ip_titulos_pub = pd.concat([ip_titulos_pub, df_linha_pct_titulos_pub])

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
ip_vendido_acoes = pd.concat([ip_vendido_acoes, df_linha_pct_vendido_acoes])

In [12]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//ip//fundo_ip_{ano}{mes}.xlsx') as writer:
    ip_acoes.to_excel(writer, sheet_name='acoes')
    ip_bdr.to_excel(writer, sheet_name='bdr')
    ip_exterior.to_excel(writer, sheet_name='exterior')
    ip_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    ip_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    ip_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

### Fundo Squadra

* https://maisretorno.com/fundo/squadra-master-long-biased-fia
* Famosos pelos shorts de:
    * IRBR - 05/2018
    * AERI3 - 01/2021
    * Nubank

In [14]:
squadra_acoes, squadra_bdr, squadra_exterior, squadra_cotas_fundos, squadra_titulos_pub, squadra_vendido_acoes = fundo_cnpj(cnpj='09.412.648/0001-40')
squadra_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//arquivo_historico//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='09.412.648/0001-40'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(squadra_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_bdr = round(pd.Series(squadra_bdr['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_exterior = round(pd.Series(squadra_exterior['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(squadra_cotas_fundos['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_titulos_pub = round(pd.Series(squadra_titulos_pub['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(squadra_vendido_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
squadra_acoes = pd.concat([squadra_acoes, df_linha_pct_acoes], ignore_index=True)

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
squadra_bdr = pd.concat([squadra_bdr, df_linha_pct_bdr], ignore_index=True)

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
squadra_exterior = pd.concat([squadra_exterior, df_linha_pct_exterior], ignore_index=True)

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
squadra_cotas_fundos = pd.concat([squadra_cotas_fundos, df_linha_pct_cotas_fundos], ignore_index=True)

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
squadra_titulos_pub = pd.concat([squadra_titulos_pub, df_linha_pct_titulos_pub], ignore_index=True)

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
squadra_vendido_acoes = pd.concat([squadra_vendido_acoes, df_linha_pct_vendido_acoes], ignore_index=True)

In [15]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//squadra//fundo_squadra_{ano}{mes}.xlsx') as writer:
    squadra_acoes.to_excel(writer, sheet_name='acoes')
    squadra_bdr.to_excel(writer, sheet_name='bdr')
    squadra_exterior.to_excel(writer, sheet_name='exterior')
    squadra_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    squadra_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    squadra_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

#### Comparação das posições compradas e vendidas

In [77]:
# Fazendo uma cópia do df original
squadra_vendido_acoes2 = squadra_vendido_acoes.copy()
# Retirando a última linha que contém a porcentagem do PL
squadra_vendido_acoes2 = squadra_vendido_acoes2.iloc[:-1]
# Selecionando apenas as colunas mais importantes
squadra_vendido_acoes2 = squadra_vendido_acoes2[['CD_ATIVO', 'VL_MERC_POS_FINAL']]
# Transformando a coluna 'CD_ATIVO' no index do df para selecionar apenas as ações que tem posição vendida
squadra_vendido_acoes2 = squadra_vendido_acoes2.set_index('CD_ATIVO')
# Renomeando a coluna 'VL_MERC_POS_FINAL'
squadra_vendido_acoes2 = squadra_vendido_acoes2.rename(columns={'VL_MERC_POS_FINAL':'VL_MERC_POS_FINAL_VENDIDO'})

# Lista das ações vendidas
lst_vendido_acoes = list(squadra_vendido_acoes['CD_ATIVO'].dropna())

# Fazendo uma cópia do df original
squadra_acoes2 = squadra_acoes.copy()
# Retirando a última linha que contém a porcentagem do PL
squadra_acoes2 = squadra_acoes2.iloc[:-1]
# Selecionando apenas as colunas mais importantes
squadra_acoes2 = squadra_acoes2[['CD_ATIVO', 'VL_MERC_POS_FINAL']]
# Transformando a coluna 'CD_ATIVO' no index do df para selecionar apenas as ações que tem posição vendida
squadra_acoes2 = squadra_acoes2.set_index('CD_ATIVO')
# Renomeando a coluna 'VL_MERC_POS_FINAL'
squadra_acoes2 = squadra_acoes2.rename(columns={'VL_MERC_POS_FINAL':'VL_MERC_POS_FINAL_COMPRADO'})
# Selecionando apenas as ações que também possuem posição de vendas
squadra_acoes2 = squadra_acoes2.loc[lst_vendido_acoes]

# Concatenando os dois dfs
comparacao_compra_venda = pd.concat([squadra_acoes2, squadra_vendido_acoes2], axis=1)
# Calculando a diferença entre a posição comprada e a posição vendida
comparacao_compra_venda['diff'] = comparacao_compra_venda['VL_MERC_POS_FINAL_COMPRADO'] - comparacao_compra_venda['VL_MERC_POS_FINAL_VENDIDO']

# Formatando as colunas com separador de milhar. Após essa formatação as colunas deixam de ser float e passam a ser str
def formatar_com_separador(valor):
    return '{:,.0f}'.format(valor)
    
comparacao_compra_venda = comparacao_compra_venda.applymap(formatar_com_separador)


comparacao_compra_venda

,VL_MERC_POS_FINAL_COMPRADO,VL_MERC_POS_FINAL_VENDIDO,diff
CD_ATIVO,,,
MGLU3,"26,825,818","56,367,350","-29,541,532"
ABEV3,"15,113,605","42,219,130","-27,105,525"
VIIA3,"7,039,650","29,849,816","-22,810,165"
PETZ3,"7,638,602","29,167,712","-21,529,111"
INTB3,"7,264,425","27,726,906","-20,462,481"
ITUB4,"17,369,277","18,906,074","-1,536,797"
WEGE3,"16,759,854","16,759,854",0
CRFB3,"3,003,518","12,692,923","-9,689,405"
